In [ ]:
'''
Project: Bangkit Capstone Project
Group: B21-CAP0194
Part: Modelling
'''

'\nProject: Bangkit Capstone Project\nGroup: B21-CAP0194\nPart: Modelling\n'

In [3]:
vocab_size = 20000
embedding_dim = 32
max_length = 32
trunc_type = 'post'
oov_tok = "<OOV>"

In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
#Import Labelled Dataset
url = 'https://drive.google.com/file/d/1nOn8OONtZA7pcX4Pvh8W_wOUg49ZWTvT/view?usp=sharing'
file_id = url.split('/')[-2]
csv_url = 'https://drive.google.com/uc?id=' + file_id
df = pd.read_csv(csv_url)
df.head()

,index,text,label
0,0,the ironies of the movement are too painfully...,private places
1,1,women in kuwait have launched a social media c...,public places
2,2,triger warning raising awareness of sexual ass...,private places
3,3,charges brought against the guy that smacked m...,private places
4,4,and the fourth presenter on 26th may is hannah...,educational places


In [ ]:
df_cut = df[:100]

In [ ]:
df_cut.head(50)

,index,text,label
0,0,the ironies of the movement are too painfully...,private places
1,1,women in kuwait have launched a social media c...,public places
2,2,triger warning raising awareness of sexual ass...,private places
3,3,charges brought against the guy that smacked m...,private places
4,4,and the fourth presenter on 26th may is hannah...,educational places
5,5,this is so messed up are there any men in the...,public places
6,6,to tackle an epidemic of male violence against...,public places
7,7,there has been a new series of harassment alle...,private places
8,8,want to see what the movement looks like its ...,public places
9,9,female students dont want to have sex w dolan ...,private places


In [ ]:
import getpass

APIKEY = getpass.getpass()

··········


In [ ]:
# running Translate API
#from googleapiclient.discovery import build
#service = build('translate', 'v2', developerKey=APIKEY)

#df_cut['indo_text'] = df_cut['text'].apply(lambda x: (service.translations().list(source='en', target='id', q=x).execute())).apply(lambda x: x['translations']).apply(lambda x: x[0]).apply(lambda x: x['translatedText'])

In [ ]:
# running Translate API
from googleapiclient.discovery import build
service = build('translate', 'v2', developerKey=APIKEY)

df['indo_text'] = df['text'].apply(lambda x: (service.translations().list(source='en', target='id', q=x).execute())).apply(lambda x: x['translations']).apply(lambda x: x[0]).apply(lambda x: x['translatedText'])

In [ ]:
df['indo_text'].head(50)

In [15]:
#Save DataFrame to CSV
df_translated = pd.DataFrame()
df_translated['index'] = df['index']
df_translated['text'] = df['text']
df_translated['indo_text'] = df['indo_text']
df_translated['label'] = df['label']
df_translated.to_csv("capstonetweettranslatednew.csv", index=False)

In [24]:
#Import Translated Dataset
url = 'https://drive.google.com/file/d/1fYLD9Po3eGJacbeWnHEo_P15w9Hd8tkS/view?usp=sharing'
file_id = url.split('/')[-2]
csv_url = 'https://drive.google.com/uc?id=' + file_id
df = pd.read_csv(csv_url)
df.head()

,index,text,indo_text,label
0,0,the ironies of the movement are too painfully...,ironi gerakan ini terlalu menyakitkan jelas di...,private places
1,1,women in kuwait have launched a social media c...,perempuan di kuwait telah meluncurkan kampanye...,public places
2,2,triger warning raising awareness of sexual ass...,peringatan triger meningkatkan kesadaran akan ...,private places
3,3,charges brought against the guy that smacked m...,dakwaan yang diajukan terhadap pria yang memuk...,private places
4,4,and the fourth presenter on 26th may is hannah...,dan presenter keempat pada tanggal 26 mei adal...,educational places


In [25]:
df['text'] = df['indo_text']
df.drop(['indo_text'], axis=1)

,index,text,label
0,0,ironi gerakan ini terlalu menyakitkan jelas di...,private places
1,1,perempuan di kuwait telah meluncurkan kampanye...,public places
2,2,peringatan triger meningkatkan kesadaran akan ...,private places
3,3,dakwaan yang diajukan terhadap pria yang memuk...,private places
4,4,dan presenter keempat pada tanggal 26 mei adal...,educational places
...,...,...,...
14103,14103,Itu peta jalan yang diuji pada implementasi la...,public places
14104,14104,serbia saya pikir saya adalah satu-satunya ber...,neutral
14105,14105,jangan bergeming bersikeras untuk memperjuangk...,public places
14106,14106,pelatihan antiharassment online gratis yang di...,private places


In [33]:
!pip install Sastrawi
!pip install swifter

     |████████████████████████████████| 634kB 6.8MB/s 
     |████████████████████████████████| 296kB 1.7MB/s 
     |████████████████████████████████| 583kB 34.1MB/s 
     |████████████████████████████████| 112kB 49.9MB/s 
     |████████████████████████████████| 48.5MB 62kB/s 
     |████████████████████████████████| 17.2MB 192kB/s 
     |████████████████████████████████| 133kB 59.6MB/s 
     |████████████████████████████████| 81kB 11.9MB/s 
     |████████████████████████████████| 204kB 53.8MB/s 
     |████████████████████████████████| 1.3MB 44.9MB/s 
     |████████████████████████████████| 81kB 362kB/s 
     |████████████████████████████████| 3.1MB 38.6MB/s 
     |████████████████████████████████| 71kB 10.7MB/s 
     |████████████████████████████████| 296kB 53.1MB/s 
     |████████████████████████████████| 143kB 58.3MB/s 
     |████████████████████████████████| 92kB 14.3MB/s 
  Created wheel for swifter: filename=swifter-1.0.7-cp37-none-any.whl size=13980 sha256=1b8f21171d64effc173fbfa3

In [26]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

df['text'] = df['text'].apply(lambda x: word_tokenize(x))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [27]:
from nltk.corpus import stopwords
nltk.download('stopwords')
list_stopwords = stopwords.words('indonesian')
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

df['text'] = df['text'].apply(stopwords_removal)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [28]:
df['text']

0        [ironi, gerakan, menyakitkan, dipelopori, aktr...
1        [perempuan, kuwait, meluncurkan, kampanye, med...
2        [peringatan, triger, meningkatkan, kesadaran, ...
3        [dakwaan, diajukan, pria, memukul, serangan, s...
4        [presenter, keempat, tanggal, 26, mei, mahasis...
                               ...                        
14103    [Itu, peta, jalan, diuji, implementasi, kerja,...
14104           [serbia, pikir, satu-satunya, berita, bbc]
14105    [bergeming, bersikeras, memperjuangkan, kejant...
14106    [pelatihan, antiharassment, online, gratis, di...
14107    [Pengacara, hak-hak, perempuan, 1, kehilangan,...
Name: text, Length: 14108, dtype: object

In [29]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter
# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in df['text']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)

def get_stemmed_term(document):
    return [term_dict[term] for term in document]

df['text'] = df['text'].swifter.apply(get_stemmed_term)

In [30]:
df['text']

0        [ironi, gera, sakit, lopor, aktris, hollywood,...
1        [perempuan, kuwait, luncur, kampanye, media, s...
2        [ingat, triger, tingkat, sadar, leceh, seksual...
3        [dakwa, aju, pria, pukul, serang, seksual, per...
4        [presenter, empat, tanggal, 26, mei, mahasiswa...
                               ...                        
14103    [itu, peta, jalan, uji, implementasi, kerja, tim]
14104                   [serbia, pikir, satu, berita, bbc]
14105    [geming, sikeras, juang, jantan, percaya, ceri...
14106    [latih, antiharassment, online, gratis, luncur...
14107    [acara, hak, perempuan, 1, hilang, tenaga, , p...
Name: text, Length: 14108, dtype: object

In [31]:
#Save DataFrame to CSV
df_stemmed = pd.DataFrame()
df_stemmed['index'] = df['index']
df_stemmed['text'] = df['text']
df_stemmed['label'] = df['label']
df_stemmed.to_csv("capstonetweetstemmed.csv", index=False)

In [58]:
#Import Translated Dataset
url = 'https://drive.google.com/file/d/1gfNBN02RnQckl_Oj_3CxhVOiWROvKXyv/view?usp=sharing'
file_id = url.split('/')[-2]
csv_url = 'https://drive.google.com/uc?id=' + file_id
df = pd.read_csv(csv_url)
df.head()

,index,text,label
0,0,"['ironi', 'gera', 'sakit', 'lopor', 'aktris', ...",private places
1,1,"['perempuan', 'kuwait', 'luncur', 'kampanye', ...",public places
2,2,"['ingat', 'triger', 'tingkat', 'sadar', 'leceh...",private places
3,3,"['dakwa', 'aju', 'pria', 'pukul', 'serang', 's...",private places
4,4,"['presenter', 'empat', 'tanggal', '26', 'mei',...",educational places


In [59]:
df['text']

0        ['ironi', 'gera', 'sakit', 'lopor', 'aktris', ...
1        ['perempuan', 'kuwait', 'luncur', 'kampanye', ...
2        ['ingat', 'triger', 'tingkat', 'sadar', 'leceh...
3        ['dakwa', 'aju', 'pria', 'pukul', 'serang', 's...
4        ['presenter', 'empat', 'tanggal', '26', 'mei',...
                               ...                        
14103    ['itu', 'peta', 'jalan', 'uji', 'implementasi'...
14104         ['serbia', 'pikir', 'satu', 'berita', 'bbc']
14105    ['geming', 'sikeras', 'juang', 'jantan', 'perc...
14106    ['latih', 'antiharassment', 'online', 'gratis'...
14107    ['acara', 'hak', 'perempuan', '1', 'hilang', '...
Name: text, Length: 14108, dtype: object

In [60]:
labels = pd.get_dummies(df.label)

In [61]:
df_new = pd.concat([df, labels], axis=1)
df_new = df_new.drop(columns='label')

In [62]:
tweet = df_new['text'].values
label = df_new[["cyber", "educational places", "neutral", "private places", "public places", "work places"]].values

In [63]:
label

array([[0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0],
       ...,
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0]], dtype=uint8)

In [64]:
from sklearn.model_selection import train_test_split
#Train Text, Test Text, Train Label, Test Label
X_train, X_test, y_train, y_test = train_test_split(tweet, label, test_size=0.2, shuffle=True)

In [65]:
X_train

array(["['pertanggungjawaban', 'menyalahgunakannya']",
       "['saya', 'aku', '03', 'salah', 'gera', 'segar', 'pikir', 'orang', '', 'percaya']",
       "['orang', 'maju', 'penting', 'wanita', 'kerja', '', 'anggota', 'undur', 'dukung', 'diam']",
       ...,
       "['ikut', 'informasi', 'tambah', 'pandu', 'dukung', 'keras', 'seksual', 'sembah', 'anda', 'koalisi', 'colorado', 'lawan', 'keras', 'seksual', 'ccasa', 'ajar', 'unduh', 'lengkap']",
       "['selasa', 'the', 'daily', 'beast', 'lapor', 'mantan', 'new', 'york', 'ag', 'eric', 'schneiderman', 'undur', 'aib', 'tuduh', 'serang', 'fisik', 'wanita', 'milik', 'ijin', 'praktek', 'hukum', '']",
       "['tindak', 'ambil', 'trudeau', 'pikir', 'momen', 'maksud', 'salah', 'alas', 'utama']"],
      dtype=object)

In [66]:
import re
X_train =  [re.sub(r"'", "", str(j)) for j in X_train]
y_train =  [re.sub(r"'", "", str(j)) for j in y_train]

In [67]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(X_train)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(X_test)
testing_padded = pad_sequences(testing_sequences, maxlen = max_length)

In [68]:
word_index

{'<OOV>': 1,
 'anda': 2,
 'wanita': 3,
 'seksual': 4,
 'leceh': 5,
 'orang': 6,
 'gera': 7,
 'saya': 8,
 'tuduh': 9,
 'milik': 10,
 'perkosa': 11,
 'pria': 12,
 'bicara': 13,
 'keras': 14,
 'salah': 15,
 'perempuan': 16,
 'laku': 17,
 'benar': 18,
 'korban': 19,
 'anak': 20,
 'dengar': 21,
 'pikir': 22,
 'dukung': 23,
 'laki': 24,
 'kerja': 25,
 'percaya': 26,
 'cerita': 27,
 'terima': 28,
 'adil': 29,
 'bantu': 30,
 'hidup': 31,
 'henti': 32,
 'polisi': 33,
 'serang': 34,
 'kuat': 35,
 'ubah': 36,
 'lapor': 37,
 'kasih': 38,
 'seks': 39,
 'bagi': 40,
 'selamat': 41,
 'hukum': 42,
 'baca': 43,
 'ya': 44,
 'temu': 45,
 'coba': 46,
 'jalan': 47,
 'dunia': 48,
 'kulit': 49,
 'hubung': 50,
 'tulis': 51,
 'harap': 52,
 'tuju': 53,
 'kali': 54,
 'hak': 55,
 'pergi': 56,
 'ambil': 57,
 'sintas': 58,
 'gadis': 59,
 'diri': 60,
 'butuh': 61,
 'lindung': 62,
 'rumah': 63,
 'buruk': 64,
 'pilih': 65,
 'acara': 66,
 'bukti': 67,
 'nyata': 68,
 'nama': 69,
 'film': 70,
 'usia': 71,
 'tolong': 72,
 

In [69]:
class myCallback(tf.keras.callbacks.Callback):
	def on_epoch_end(self, epoch, logs={}):
		if(logs.get('val_accuracy')>0.90):
			print("Desired accuracy reached!")
			self.model.stop_training=True

In [ ]:
import tensorflow as tf
'''
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
  #tf.keras.layers.Flatten(),
  tf.keras.layers.LSTM(64),
  tf.keras.layers.Dense(32, activation='relu'),
  tf.keras.layers.Dense(6, activation='softmax')
])

'''
'''
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
  #tf.keras.layers.Flatten(),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
  tf.keras.layers.Dense(32, activation='relu'),
  tf.keras.layers.Dense(6, activation='softmax')
])
'''
'''
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
  #tf.keras.layers.Flatten(),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16)),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(32, activation='relu'),
  tf.keras.layers.Dense(6, activation='softmax')
])
'''
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
  #tf.keras.layers.Flatten(),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(32, activation='relu'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(16, activation='relu'),
  tf.keras.layers.Dense(6, activation='softmax')
])

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 32, 32)            640000    
_________________________________________________________________
bidirectional_8 (Bidirection (None, 32, 256)           164864    
_________________________________________________________________
bidirectional_9 (Bidirection (None, 32, 128)           164352    
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 64)                41216     
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_6 (Dropout)          (None, 32)               

In [ ]:
num_epochs = 100
callbacks = myCallback()
history = model.fit(padded, y_train, epochs=num_epochs, validation_data=(testing_padded, y_test), callbacks=[callbacks])

Epoch 1/100
309/309 [==============================] - 17s 31ms/step - loss: 1.3967 - accuracy: 0.3837 - val_loss: 1.0992 - val_accuracy: 0.5519
Epoch 2/100
309/309 [==============================] - 8s 25ms/step - loss: 0.9942 - accuracy: 0.6179 - val_loss: 0.9007 - val_accuracy: 0.6688
Epoch 3/100
309/309 [==============================] - 8s 25ms/step - loss: 0.7426 - accuracy: 0.7327 - val_loss: 0.8888 - val_accuracy: 0.6927
Epoch 4/100
309/309 [==============================] - 8s 26ms/step - loss: 0.6164 - accuracy: 0.7909 - val_loss: 0.9297 - val_accuracy: 0.6995
Epoch 5/100
309/309 [==============================] - 8s 25ms/step - loss: 0.4863 - accuracy: 0.8401 - val_loss: 0.9671 - val_accuracy: 0.7002
Epoch 6/100
309/309 [==============================] - 8s 25ms/step - loss: 0.3799 - accuracy: 0.8776 - val_loss: 0.8912 - val_accuracy: 0.7052
Epoch 7/100
309/309 [==============================] - 8s 25ms/step - loss: 0.3075 - accuracy: 0.9009 - val_loss: 0.9459 - val_accuracy

In [ ]:
text = ["jimmy rape me in my house"]

tokenized_text = tokenizer.texts_to_sequences(text)

In [ ]:
prediction = model.predict(tokenized_text)

In [ ]:
labels = ["cyber", "educational places", "neutral", "private places", "public places", "work places"]
predicted = prediction.argmax(axis=1)
predict_labels = [labels[predict] for predict in predicted]
print(prediction)
print(predict_labels)

[[2.9312835e-08 2.7396174e-10 1.3652676e-05 5.5923681e-03 9.9438465e-01
  9.2011460e-06]]
['public places']


In [ ]:
#Exporting the Model (TFLITE)
export_dir = 'saved_model/1'
tf.saved_model.save(model, export_dir)

INFO:tensorflow:Assets written to: saved_model/1/assets


INFO:tensorflow:Assets written to: saved_model/1/assets


In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
#converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

In [ ]:
import pathlib

tflite_model_file = pathlib.Path('/content/model.tflite')
tflite_model_file.write_bytes(tflite_model)

4106416

In [ ]:
from google.colab import files
files.download(tflite_model_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Exporting the Tokenized Word
import json

with open( 'word_dict.json' , 'w' ) as file:
    json.dump( tokenizer.word_index , file )